<a href="https://colab.research.google.com/github/DaeSeokSong/LSTM-PPoA/blob/main/Predict_Price_of_Agricultural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참조문헌
## [A Prediction Model for Agricultural Products Price with LSTM Network](https://www.koreascience.or.kr/article/JAKO201809469053682.page)

**신성호, 이미경, 송사광**

한국과학기술정보연구원 연구데이터플랫폼센터,
한국과학기술정보연구원 연구데이터플랫폼센터/과학기술연합대학원대학교 빅데이터과학과

Sungho Shin(maximus74@kisti.re.kr), Mikyoung Lee(jerryis@kisti.re.kr),
Sa-kwang Song(esmallj@kisti.re.kr)

# 당년 농산물 가격 예측 LSTM 모델

Input = 1~저번 달까지의 pram 값을 하나로 묶은 array(인스턴스)

output = 당월의 해당 채소 가격

layer = 원래 해당 채소 가격

### Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/PPoA
!ls -al

Mounted at /content/gdrive
/content/gdrive/MyDrive/DeepLearning/Project/PPoA
total 26
drwx------ 2 root root  4096 Nov 24 06:07  Dataset
-rw------- 1 root root 21558 Nov 25 10:31 'Predict Price of Agricultural.ipynb'


### Import


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.utils import *
from sklearn.preprocessing import *

### Function

In [3]:
def Normalizer(targetData) :
    return (targetData - targetData.min()) / ( targetData.max() - targetData.min())

### Global variable

In [4]:
START_YEAR = 0
END_YEAR = 0

### Data read

In [24]:
START_YEAR = 2006
END_YEAR = 2020
dataset = []

for next in range(0, (END_YEAR-START_YEAR)+1) :
    cur_year = START_YEAR + next

    f_name = str(cur_year)+'_onion.csv'
    df = pd.read_csv('./Dataset/'+f_name, index_col=0)

    ''' 기상변수 '''
    # 강수량
    precipi_avg = df['평균월강수량(mm)']
    precipi_max = df['최다월강수량(mm)']
    # 기온
    temper_avg = df['평균기온(℃)']
    temper_max = df['평균최고기온(℃)']
    temper_min = df['평균최저기온(℃)']
    # 풍속
    windSpeed_avg = df['평균풍속(m/s)']
    windSpeed_max = df['최대풍속(m/s)']
    # 습도
    humidity_avg = df['평균습도(%rh)']
    humidity_min = df['최저습도(%rh)']
    # 일조량 / 일사량
    sunshine = df['일조합']
    insolation = df['일사합']

    ''' 기타변수 '''
    # 전년 재배면적
    last_cultiv_area = df['전년생산면적']
    # 전년 평균 생산량(수확량)
    last_production = df['전년생산량']
    # 전년 평균 생산단수
    last_prod_unit = df['전년생산단수']
    # 전년 수입량
    last_amount_import = df['전년수입량']
    # 해당 농작물 가격
    crops_price = df['가격']
    # 경유 가격
    diesel_price = df['경유가격']
    # 물가지수(price index -> pidx), 2015년 기준 얼마나 오르고 내렸는지
    total_pidx = df['총물가지수']
    prod_pidx = df['상품']
    agricul_marine_prod_pidx = df['농축수산물']
    indust_prod_pidx = df['공업제품']
    serv_pidx = df['서비스']
    pub_serv_pidx = df['공공서비스']
    per_serv_pidx = df['개인서비스']
    house_pidx = df['집세']

    # 정규화 해야하는 데이터셋
    norm_prams = [precipi_avg, precipi_max,              # 강수량
                  temper_avg, temper_max, temper_min,    # 기온
                  windSpeed_avg, windSpeed_max,          # 풍속
                  humidity_avg, humidity_min,            # 습도
                  sunshine,                              # 일조량
                  insolation,                            # 일사량
                  crops_price,                           # 해당 농작물 월별 가격
                  diesel_price,                          # 월별 경유 가격
                  # 물가지수
                  total_pidx, prod_pidx, agricul_marine_prod_pidx, indust_prod_pidx, serv_pidx, pub_serv_pidx, per_serv_pidx, house_pidx,
                  last_amount_import                    # 전년 수입량
                ]
    # 정규화 안 하는 데이터셋
    denorm_prams = [last_cultiv_area,                      # 전년 재배면적
                    last_production,                       # 전년 평균 생산량
                    last_prod_unit                        # 전년 평균 생산단수
                    ]

    dataset.append([norm_prams, denorm_prams])

### Data preprocessing

In [25]:
# 정규화
for idx, data in enumerate(dataset) :
    for i, pram in enumerate(data[0]) :
        pram = Normalizer(pram)
        pram = pram.to_numpy()

        dataset[idx][0][i] = pram

[[[array([0.02398082, 0.027498  , 0.        , 0.14148681, 0.23245404,
       0.21518785, 1.        , 0.17985612, 0.07258193, 0.05259792,
       0.05947242, 0.01486811]), array([0.02808417, 0.03122349, 0.        , 0.1840319 , 0.22662481,
       0.35643984, 1.        , 0.1948074 , 0.17571695, 0.23358222,
       0.13965722, 0.08111319]), array([0.        , 0.02316602, 0.2046332 , 0.41312741, 0.64478764,
       0.7992278 , 0.87258687, 1.        , 0.72972973, 0.62548263,
       0.32046332, 0.05405405]), array([0.        , 0.03088803, 0.25096525, 0.43243243, 0.66409266,
       0.81081081, 0.81081081, 1.        , 0.74903475, 0.69111969,
       0.33204633, 0.07335907]), array([0.        , 0.00377358, 0.14339623, 0.36981132, 0.60377358,
       0.77735849, 0.92075472, 1.        , 0.70943396, 0.58113208,
       0.29056604, 0.05283019]), array([0.33333333, 0.75      , 0.91666667, 1.        , 0.5       ,
       0.25      , 0.25      , 0.08333333, 0.25      , 0.        ,
       0.5       , 0.3333333

### Modeling

In [ ]:
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=?
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.01)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='rmsprop')

### Learning

### Prediction